In [188]:
from sqlalchemy import create_engine, Column, Integer, String, DateTime, ForeignKey, Enum, Float
from sqlalchemy.orm import sessionmaker, declarative_base, validates
import pandas as pd

# 创建数据库连接引擎和会话工厂
engine = create_engine('sqlite:///products.db')
Session = sessionmaker(bind=engine)

# 创建模型基类
Base = declarative_base()

# 定义商品模型类
class Product(Base):
    __tablename__ = 'products'

    id = Column(String, primary_key=True)
    product_ts = Column(DateTime, nullable=False)
    name = Column(String(256), nullable=False)
    color = Column(String(64), nullable=False)
    size = Column(String(64), nullable=False)
    photo_path = Column(String(256), nullable=True)

    @validates('color')
    def validate_color(self, key, color):
        allowed_colors = ["紅", "綠", "藍", "黃", "黑", "灰"]
        if color not in allowed_colors:
            raise ValueError(f"invalid color, allowed values are {allowed_colors}")
        return color

    @validates('size')
    def validate_size(self, key, size):
        allowed_sizes = ["S", "M", "L", "2L", "XL", "XXL"]
        if size not in allowed_sizes:
            raise ValueError(f"invalid size, allowed values are {allowed_sizes}")
        return size

class Detail(Base):
    __tablename__ = 'detail'

    id = Column(String, ForeignKey('products.id'), primary_key=True)
    detail_ts = Column(DateTime, default=DateTime, primary_key=True)
    quantity = Column(Float, nullable=False)
    price = Column(Float, nullable=False)
    type = Column(Enum('IN', 'OUT', name='type'), nullable=False)
    supplier = Column(String(256), nullable=True)
    note = Column(String(256), nullable=True)


# 创建数据表
Base.metadata.create_all(engine)

# 创建商品记录的函数
def create_product(name, color, size, photo_path=None):
    id = f'{name}_{color}_{size}'
    session = Session()
    product = Product(id=id, product_ts=pd.Timestamp.now(), name=name, color=color, size=size, photo_path=photo_path)
    session.add(product)
    session.commit()
    return product

# 根据商品 ID 查找商品记录的函数
def read_product(id):
    session = Session()
    product = session.query(Product).filter_by(id=id).first()
    session.commit()
    return product

# 更新商品记录的函数
def update_product(id, name=None, color=None, size=None, photo_path=None):
    session = Session()
    product = session.query(Product).filter_by(id=id).first()
    if name:
        product.name = name
    if color:
        product.color = color
    if size:
        product.size = size
    if photo_path:
        product.photo_path = photo_path
    session.commit()
    return product

# 删除商品记录的函数
def delete_product(id):
    session = Session()
    product = session.query(Product).filter_by(id=id).first()
    session.delete(product)
    session.commit()
    return product

def get_product_data():
    with Session() as s:
        results = s.query(Product).all()
    df = pd.DataFrame([(p.id, p.name, p.color, p.size, p.photo_path, p.product_ts) for p in results],
                      columns=['id', 'name', 'color', 'size', 'photo_path', 'product_ts'])
    return df

# 创建商品记录的函数
def create_product＿detail(id, quantity, price, type, supplier, note=None):
    session = Session()
    detail = Detail(detail_ts=pd.Timestamp.now() - pd.Timedelta(pd.Series(range(3000)).sample(1).values[0], 'day'), id=id, quantity=quantity, price=price, type=type, supplier=supplier, note=note)
    session.add(detail)
    session.commit()
    return detail

# 根据商品 ID 查找商品记录的函数
def read_product_detail(id):
    with Session() as s:
        results = s.query(Detail).filter_by(id=id).all()
    df = pd.DataFrame([(d.id, d.detail_ts, d.quantity, d.price, d.type, d.supplier, d.note) for d in results], 
                      columns=['id', 'detail_ts', 'quantity', 'price', 'type', 'supplier', 'note'])
    return df

def get_detail_data():
    with Session() as s:
        results = s.query(Detail).all()
    df = pd.DataFrame([(d.id, d.detail_ts, d.quantity, d.price, d.type, d.supplier, d.note) for d in results], 
                      columns=['id', 'detail_ts', 'quantity', 'price', 'type', 'supplier', 'note'])
    return df.sort_values('detail_ts')


# Base.metadata.drop_all(engine)

In [113]:
colors = ["紅", "綠", "藍", "黃", "黑", "灰"]
size = ["S", "M", "L", "2L", "XL", "XXL"]

for c in colors:
    for s in size:
        for pt in ['男褲', '女瑜珈褲', '中性壓力褲']:
            create_product(pt, c, s)

# create_product('女裝', '黑', 'color')

In [189]:
quantity = range(50, 201, 70)
type = ['IN', 'OUT']
supplier = ['柏國', '蝦皮']

r = 0
for _ in range(10):
    for i in get_product_data().id.sample(100).values:
        for t in type:
            for s in supplier:
                m = 3 if t == 'IN' else -1
                create_product＿detail(i, pd.Series([50,150,250]).sample(1).values[0] * m, 500 + pd.Series([100,500,1500]).sample(1).values[0], t, s)

In [103]:
get_product_data()

,id,name,color,size,photo_path,product_ts
0,男褲_紅_S,男褲,紅,S,None,2023-05-10 20:53:06.777552
1,女瑜珈褲_紅_S,女瑜珈褲,紅,S,None,2023-05-10 20:53:06.786553
2,中性壓力褲_紅_S,中性壓力褲,紅,S,None,2023-05-10 20:53:06.789553
3,男褲_紅_M,男褲,紅,M,None,2023-05-10 20:53:06.792552
4,女瑜珈褲_紅_M,女瑜珈褲,紅,M,None,2023-05-10 20:53:06.796553
...,...,...,...,...,...,...
103,女瑜珈褲_灰_XL,女瑜珈褲,灰,XL,None,2023-05-10 20:53:07.122555
104,中性壓力褲_灰_XL,中性壓力褲,灰,XL,None,2023-05-10 20:53:07.125555
105,男褲_灰_XXL,男褲,灰,XXL,None,2023-05-10 20:53:07.128551
106,女瑜珈褲_灰_XXL,女瑜珈褲,灰,XXL,None,2023-05-10 20:53:07.131552


In [190]:
from sqlalchemy import select

def get_detail_with_product_info():
    with Session() as s:
        results = s.query(Detail).all()
    df = pd.DataFrame([(d.id, d.detail_ts, d.quantity, d.price, d.type, d.supplier, d.note) for d in results], 
                      columns=['id', 'detail_ts', 'quantity', 'price', 'type', 'supplier', 'note'])
    return df.sort_values('detail_ts')

df = get_detail_data()
df

,id,detail_ts,quantity,price,type,supplier,note
2994,男褲_黑_L,2015-02-24 23:32:35.699378,-50.0,1000.0,OUT,柏國,None
4465,女瑜珈褲_黃_L,2015-02-24 23:32:41.263774,450.0,2000.0,IN,蝦皮,None
5096,中性壓力褲_綠_L,2015-02-24 23:32:45.125192,150.0,1000.0,IN,柏國,None
3404,男褲_綠_2L,2015-02-26 23:32:37.221376,750.0,2000.0,IN,柏國,None
3699,女瑜珈褲_灰_M,2015-02-26 23:32:38.338376,-150.0,2000.0,OUT,蝦皮,None
...,...,...,...,...,...,...,...
410,女瑜珈褲_灰_L,2023-05-09 23:31:18.950836,-250.0,600.0,OUT,柏國,None
570,中性壓力褲_黑_2L,2023-05-09 23:31:19.610426,-50.0,600.0,OUT,柏國,None
799,中性壓力褲_綠_S,2023-05-09 23:31:42.628877,-50.0,600.0,OUT,蝦皮,None
1964,男褲_藍_2L,2023-05-09 23:32:31.485673,450.0,1000.0,IN,柏國,None


In [191]:
v = df.groupby('id').quantity.sum()
p = df.groupby('id').apply(lambda x:x.price[x.quantity>0].mean())
o = df.groupby('id').apply(lambda x:x.price[x.quantity<0].mean())
p
o

id
中性壓力褲_灰_2L    1384.615385
中性壓力褲_灰_L     1381.818182
中性壓力褲_灰_M     1020.000000
中性壓力褲_灰_S     1458.333333
中性壓力褲_灰_XL    1044.444444
                 ...     
男褲_黑_L        1183.333333
男褲_黑_M         900.000000
男褲_黑_S        1200.000000
男褲_黑_XL       1241.666667
男褲_黑_XXL      1172.727273
Length: 108, dtype: float64

In [192]:
ret = df.groupby(['id', 'type']).agg({'quantity': 'sum', 'price': 'mean'})
ret.columns = ['進出數量', '平均成本']
ret['總金額'] = ret['進出數量'] * ret['平均成本'].round()
ret.reset_index()
pd.merge(ret.query('type == "IN"').reset_index(), ret.query('type == "OUT"').reset_index(), on='id', how='outer').fillna(0)


,id,type_x,進出數量_x,平均成本_x,總金額_x,type_y,進出數量_y,平均成本_y,總金額_y
0,中性壓力褲_灰_2L,IN,10200.0,1092.307692,11138400.0,OUT,-4200.0,1384.615385,-5817000.0
1,中性壓力褲_灰_L,IN,12000.0,1381.818182,16584000.0,OUT,-3600.0,1381.818182,-4975200.0
2,中性壓力褲_灰_M,IN,9000.0,1120.000000,10080000.0,OUT,-3300.0,1020.000000,-3366000.0
3,中性壓力褲_灰_S,IN,10500.0,1208.333333,12684000.0,OUT,-3600.0,1458.333333,-5248800.0
4,中性壓力褲_灰_XL,IN,8100.0,1211.111111,9809100.0,OUT,-3100.0,1044.444444,-3236400.0
...,...,...,...,...,...,...,...,...,...
103,男褲_黑_L,IN,9000.0,1158.333333,10422000.0,OUT,-4000.0,1183.333333,-4732000.0
104,男褲_黑_M,IN,9600.0,1180.000000,11328000.0,OUT,-3000.0,900.000000,-2700000.0
105,男褲_黑_S,IN,6600.0,1166.666667,7702200.0,OUT,-2900.0,1200.000000,-3480000.0
106,男褲_黑_XL,IN,9600.0,1050.000000,10080000.0,OUT,-3000.0,1241.666667,-3726000.0


In [202]:
import vectorbt as vbt
d = df.groupby(['id', df.detail_ts.dt.date]).quantity.sum().reset_index().query('id == "中性壓力褲_灰_2L"').set_index('detail_ts').quantity
# pd.Series(d.values).vbt.plot()
fig = pd.Series(d.values).vbt.barplot()
pd.Series(d.values).cumsum().vbt.plot(fig=fig)

AttributeError: type object 'DOMWidget' has no attribute '_ipython_display_'

FigureWidget({
    'data': [{'showlegend': False,
              'type': 'bar',
              'uid': '3b8aa52e-8ae5-4c3a-9cd8-1b8ecf720f04',
              'x': array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
                          18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
                          36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50], dtype=int64),
              'y': array([-250.,  150.,  450.,  450.,  750.,  150.,  750., -250.,  150.,  150.,
                           -50.,  -50., -250.,  150.,  150., -250., -250.,  150.,  -50.,  -50.,
                           -50.,  450., -150., -150.,  750.,  150., -250., -150.,  450.,  150.,
                           750., -150.,  -50., -150., -250.,  450.,  450., -250., -150.,  150.,
                           150., -250.,  -50.,  450.,  450., -500.,  750., -150.,  750.,  450.,
                           -50.])},
             {'showlegend': False,
              'type': 'scatter',
              'uid': '61bebd35-a687-4230-9947-3cebb03cfca2',
              'x': array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
                          18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
                          36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50], dtype=int64),
              'y': array([-250., -100.,  350.,  800., 1550., 1700., 2450., 2200., 2350., 2500.,
                          2450., 2400., 2150., 2300., 2450., 2200., 1950., 2100., 2050., 2000.,
                          1950., 2400., 2250., 2100., 2850., 3000., 2750., 2600., 3050., 3200.,
                          3950., 3800., 3750., 3600., 3350., 3800., 4250., 4000., 3850., 4000.,
                          4150., 3900., 3850., 4300., 4750., 4250., 5000., 4850., 5600., 6050.,
                          6000.])}],
    'layout': {'height': 350,
               'legend': {'orientation': 'h',
                          'traceorder': 'normal',
                          'x': 1,
                          'xanchor': 'right',
                          'y': 1.02,
                          'yanchor': 'bottom'},
               'margin': {'b': 30, 'l': 30, 'r': 30, 't': 30},
               'template': '...',
               'width': 700}
})